<a href="https://colab.research.google.com/github/chasdabigone/RoD_Equipment_Search/blob/main/rod_ascii_creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 1. Upload your file to the gui on the left as 'image.jpg'
# 2. Click runtime->run all
# 3. picture.html (viewable with browser) and picture.txt (ascii) will be output as files
# 4. adjust columns slider to change width of picture (must re-run program)

In [ ]:
pip install ascii_magic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.2/733.2 kB 14.2 MB/s eta 0:00:00


In [ ]:
from ascii_magic import AsciiArt, Back

# Change columns to adjust width of picture. 80 is max
columns = 80 #@param {type: "slider", min: 20, max: 80}


widthRatio = 2.2
def create():
  global widthRatio
  my_art = AsciiArt.from_image('/content/image.jpg')
  my_art.to_html_file('/content/picture.html',columns=columns,width_ratio=widthRatio)
  # include back=Back.WHITE or any other primary color to change background color (does not work for html file)
  my_art.to_file('/content/picture.txt',columns=columns,width_ratio=widthRatio)

  with open(r"/content/picture.txt", 'r') as fp:
      x = len(fp.readlines())
      if x != 24:
        widthRatio = widthRatio * x / 24
        create()

create()
